# Data Cleaning, Star Schema Construction, and Metrics Development: A Practical Guide with Power BI Visualization

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

The following project consists in transforming a transactions dataset into an interactive dashboard with the cleaned data and the main findings.

**Objective:** transforming raw data into valuable insights.

**Main tools:** Python, SQL, and Power BI.


In case you want to directly see the dashboard, please click the Power BI logo below:

<a href="https://app.powerbi.com/view?r=eyJrIjoiOGE2NjNlOTQtZWU1Zi00YjQ5LTgwYjEtMWQzMzQyMThmYmY5IiwidCI6ImY2ZjdhNzFkLTE1ZDYtNGUwNC04ZjkzLTYwNzgyNmIzYjhmYyIsImMiOjR9&embedImagePlaceholder=true">
    <img src="https://drive.google.com/thumbnail?id=15lQgKjG5XmadP6pr2W-anwcCzO3eqY_K" alt="Dashboard" width="200">
</a>

This showcases the dataset's information, and the main metrics extracted from it, with an interactive dashboard that allows the user to filter the data by different categories, and visualize all the important information in one place.

## Part I: Data cleaning with pandas

We're going to use mainly the library pandas to identify and fill the missing values, remove duplicates, and remove invalid characters if found

In [2]:
import numpy as np
import pandas as pd

### Exploring the dataset

In [5]:
transactions=pd.read_csv("Transactions data set.csv")
# pd.set_option('display.max_rows', None)
transactions.head()

# We can explore the whole dataset visually setting the max rows display to None and calling the dataset


,CLIENT_ID,CLIENT_NAME,CLIENT_LASTNAME,EMAIL,STORE_ID,STORE_NAME,LOCATION,PRODUCT_ID,PRODUCT_NAME,CATEGORY,BRAND,ADDRESS_ID,STREET,CITY,STATE,ZIP_CODE,TRANSACTION_ID,QUANTITY_OF_ITEMS_SOLD,UNIT_PRICE,DISCOUNT
0,34,Client_34,Lastname_34,client_34.lastname_34@example.com,109,Store_109,Location_109,204,Product_204,Electronics,Brand_204,305,Street_305,City_305,State_305,73202,1108,5,51.258026,0.17
1,7,Client_7,Lastname_7,client_7.lastname_7@example.com,103,Store_103,Location_103,201,Product_201,Electronics,Brand_201,302,Street_302,City_302,State_302,76878,1004,9,63.984961,0.25
2,46,Client_46,Lastname_46,client_46.lastname_46@example.com,108,Store_108,Location_108,205,Product_205,Electronics,Brand_205,308,Street_308,City_308,State_308,65936,1185,8,74.113977,0.28
3,10,Client_10,Lastname_10,client_10.lastname_10@example.com,101,Store_101,Location_101,210,Product_210,Fashion,Brand_210,305,Street_305,City_305,State_305,34624,1008,9,69.335310,0.01
4,2,Client_2,Lastname_2,client_2.lastname_2@example.com,104,Store_104,Location_104,206,Product_206,Home,Brand_206,306,Street_306,City_306,State_306,47351,1004,4,70.504220,0.11


In [19]:
transactions.info()

# Or we can explore the dataset with methods, which let us identify 13 NAs in 'CLIENT_NAME', 14 in 'STORE_NAME' and 14 in "PRODUCT_NAME"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   CLIENT_ID               220 non-null    int64  
 1   CLIENT_NAME             207 non-null    object 
 2   CLIENT_LASTNAME         220 non-null    object 
 3   EMAIL                   220 non-null    object 
 4   STORE_ID                220 non-null    int64  
 5   STORE_NAME              206 non-null    object 
 6   LOCATION                220 non-null    object 
 7   PRODUCT_ID              220 non-null    int64  
 8   PRODUCT_NAME            206 non-null    object 
 9   CATEGORY                220 non-null    object 
 10  BRAND                   220 non-null    object 
 11  ADDRESS_ID              220 non-null    int64  
 12  STREET                  220 non-null    object 
 13  CITY                    220 non-null    object 
 14  STATE                   220 non-null    ob

Let's see how many rows we have to correct: 

In [7]:
print(len(transactions) - len(transactions.dropna())) # 35 rows with missing values
print(len(transactions) - len(transactions.drop_duplicates())) # 20 duplicated records


35
20


We also saw that the data is not ordered, so we can order it before cleaning it to have a more readable dataset to work with

In [8]:
transactions.sort_values(["CLIENT_ID", "STORE_ID", "PRODUCT_ID"], ignore_index=True, inplace=True)
transactions.head()
# Now we can further explore it already sorted


,CLIENT_ID,CLIENT_NAME,CLIENT_LASTNAME,EMAIL,STORE_ID,STORE_NAME,LOCATION,PRODUCT_ID,PRODUCT_NAME,CATEGORY,BRAND,ADDRESS_ID,STREET,CITY,STATE,ZIP_CODE,TRANSACTION_ID,QUANTITY_OF_ITEMS_SOLD,UNIT_PRICE,DISCOUNT
0,1,Client_1,Lastname_1,client_1.lastname_1@example.com,101,Store_101,Location_101,205,Product_205,Electronics,Brand_205,301,Street_301,City_301,State_301,36902,1098,1,20.774831,0.17
1,1,Client_1,Lastname_1,client_1.lastname_1@example.com,102,Store_102,Location_102,206,Product_206,Home,Brand_206,309,Street_309,City_309,State_309,66737,1105,3,70.747518,0.14
2,1,Client_1,Lastname_1,client_1.lastname_1@example.com,103,Store_103,Location_103,208,Product_208,Home,Brand_208,309,Street_309,City_309,State_309,82521,1085,7,59.612713,0.30
3,1,Client_1,Lastname_1,client_1.lastname_1@example.com,103,Store_103,Location_103,209,Product_209,Home,Brand_209,301,Street_301,City_301,State_301,57070,1116,7,84.130312,0.27
4,1,Client_1,Lastname_1,client_1.lastname_1@example.com,105,Store_105,Location_105,204,Product_204,Fashion,Brand_204,302,Street_302,City_302,State_302,48366,1126,1,81.400697,0.04


### Deleting duplicates

From now on, we'll leave the original ordered but otherwise unchanged, and make a copy to safely keep the original dataset

In [ ]:
# pandas already has a method that lets us easily remove duplicate records

transactions2 = transactions.drop_duplicates()
len(transactions2)
transactions2.sort_values(["CLIENT_ID", "STORE_ID", "PRODUCT_ID"], ignore_index=True, inplace=True)

# we can also sort again to avoid having missing indexes

&nbsp;


### Identifying and filling missing values

In [10]:
missingrows = np.where(transactions2.isna().any(axis=1))
transactions2.loc[missingrows]

# this lets us directly see just the rows with missing values


,CLIENT_ID,CLIENT_NAME,CLIENT_LASTNAME,EMAIL,STORE_ID,STORE_NAME,LOCATION,PRODUCT_ID,PRODUCT_NAME,CATEGORY,BRAND,ADDRESS_ID,STREET,CITY,STATE,ZIP_CODE,TRANSACTION_ID,QUANTITY_OF_ITEMS_SOLD,UNIT_PRICE,DISCOUNT
14,3,Client_3,Lastname_3,client_3.lastname_3@example.com,106,Store_106,Location_106,206,NaN,Fashion,Brand_206,310,Street_310,City_310,State_310,37362,1063,1,99.257141,0.26
19,4,Client_4,Lastname_4,client_4.lastname_4@example.com,103,NaN,Location_103,208,Product_208,Electronics,Brand_208,306,Street_306,City_306,State_306,97934,1175,3,10.153771,0.25
23,5,Client_5,Lastname_5,client_5.lastname_5@example.com,101,NaN,Location_101,202,NaN,Electronics,Brand_202,306,Street_306,City_306,State_306,25036,1199,9,96.367446,0.25
51,12,NaN,Lastname_12,client_12.lastname_12@example.com,102,NaN,Location_102,206,Product_206,Electronics,Brand_206,303,Street_303,City_303,State_303,47711,1051,9,15.967384,0.09
56,14,NaN,Lastname_14,client_14.lastname_14@example.com,101,Store_101,Location_101,201,Product_201,Electronics,Brand_201,302,Street_302,City_302,State_302,17472,1029,4,75.263390,0.22
59,14,NaN,Lastname_14,client_14.lastname_14@example.com,104,Store_104,Location_104,203,Product_203,Fashion,Brand_203,301,Street_301,City_301,State_301,80437,1071,2,32.308435,0.16
61,14,NaN,Lastname_14,client_14.lastname_14@example.com,107,Store_107,Location_107,205,Product_205,Fashion,Brand_205,306,Street_306,City_306,State_306,53476,1042,1,56.764638,0.05
70,17,NaN,Lastname_17,client_17.lastname_17@example.com,104,Store_104,Location_104,209,Product_209,Fashion,Brand_209,303,Street_303,City_303,State_303,64969,1044,5,61.550408,0.16
76,19,NaN,Lastname_19,client_19.lastname_19@example.com,104,Store_104,Location_104,206,Product_206,Electronics,Brand_206,309,Street_309,City_309,State_309,18828,1002,3,89.662556,0.11
78,19,NaN,Lastname_19,client_19.lastname_19@example.com,105,Store_105,Location_105,208,Product_208,Electronics,Brand_208,307,Street_307,City_307,State_307,22210,1198,10,48.721281,0.03


While we could fill every missing value individually, the missing values have a pattern, so using functions is more convenient and scalable. 

So, I made a function that evaluates row by row to find a missing value, and then corrects it according to the information in the rest of the dataset.

In [11]:
transactions3 = transactions2 # We first make a copy to keep having checkpoints in the dataset
transactions3 = transactions3.convert_dtypes() # We also convert to dtypes, which converts the dataframe columns to dtypes that support pd.NA


def fill_client_name(row):
    if pd.isna(row['CLIENT_NAME']):
        return 'Client_' + str(row['CLIENT_ID'])
    else:
        return row['CLIENT_NAME']

def fill_store_name(row):
    if pd.isna(row['STORE_NAME']):
        return 'Store_' + str(row['STORE_ID'])
    else:
        return row['STORE_NAME']

def fill_product_name(row):
    if pd.isna(row['PRODUCT_NAME']):
        return 'Product_' + str(row['PRODUCT_ID'])
    else:
        return row['PRODUCT_NAME']


The previous functions take the datasets' rows as arguments, provided by the "axis=1" argument in the apply method. 

The function first evaluates if the value is NaN (missing), and if that yields true, it changes the value to the correct construction. If the value is not NaN, it just returns the value stored, leaving it as it is.

In [12]:
transactions3['CLIENT_NAME'] = transactions3.apply(fill_client_name, axis=1)
transactions3['STORE_NAME'] = transactions3.apply(fill_store_name, axis=1)
transactions3['PRODUCT_NAME'] = transactions3.apply(fill_product_name, axis=1)

# We apply each function to its respective column

In [52]:
transactions3.loc[missingrows]

,CLIENT_ID,CLIENT_NAME,CLIENT_LASTNAME,EMAIL,STORE_ID,STORE_NAME,LOCATION,PRODUCT_ID,PRODUCT_NAME,CATEGORY,BRAND,ADDRESS_ID,STREET,CITY,STATE,ZIP_CODE,TRANSACTION_ID,QUANTITY_OF_ITEMS_SOLD,UNIT_PRICE,DISCOUNT
14,3,Client_3,Lastname_3,client_3.lastname_3@example.com,106,Store_106,Location_106,206,Product_206,Fashion,Brand_206,310,Street_310,City_310,State_310,37362,1063,1,99.257141,0.26
19,4,Client_4,Lastname_4,client_4.lastname_4@example.com,103,Store_103,Location_103,208,Product_208,Electronics,Brand_208,306,Street_306,City_306,State_306,97934,1175,3,10.153771,0.25
23,5,Client_5,Lastname_5,client_5.lastname_5@example.com,101,Store_101,Location_101,202,Product_202,Electronics,Brand_202,306,Street_306,City_306,State_306,25036,1199,9,96.367446,0.25
51,12,Client_12,Lastname_12,client_12.lastname_12@example.com,102,Store_102,Location_102,206,Product_206,Electronics,Brand_206,303,Street_303,City_303,State_303,47711,1051,9,15.967384,0.09
56,14,Client_14,Lastname_14,client_14.lastname_14@example.com,101,Store_101,Location_101,201,Product_201,Electronics,Brand_201,302,Street_302,City_302,State_302,17472,1029,4,75.26339,0.22
59,14,Client_14,Lastname_14,client_14.lastname_14@example.com,104,Store_104,Location_104,203,Product_203,Fashion,Brand_203,301,Street_301,City_301,State_301,80437,1071,2,32.308435,0.16
61,14,Client_14,Lastname_14,client_14.lastname_14@example.com,107,Store_107,Location_107,205,Product_205,Fashion,Brand_205,306,Street_306,City_306,State_306,53476,1042,1,56.764638,0.05
70,17,Client_17,Lastname_17,client_17.lastname_17@example.com,104,Store_104,Location_104,209,Product_209,Fashion,Brand_209,303,Street_303,City_303,State_303,64969,1044,5,61.550408,0.16
76,19,Client_19,Lastname_19,client_19.lastname_19@example.com,104,Store_104,Location_104,206,Product_206,Electronics,Brand_206,309,Street_309,City_309,State_309,18828,1002,3,89.662556,0.11
78,19,Client_19,Lastname_19,client_19.lastname_19@example.com,105,Store_105,Location_105,208,Product_208,Electronics,Brand_208,307,Street_307,City_307,State_307,22210,1198,10,48.721281,0.03


We can check that the previously missing values have all been filled correctly, with a scalable and flexible function that would allow us to work with large and complex datasets!

Finally, we can export the cleaned dataset to work with it without worrying about faulty data.

In [13]:
transactions3.tail()

,CLIENT_ID,CLIENT_NAME,CLIENT_LASTNAME,EMAIL,STORE_ID,STORE_NAME,LOCATION,PRODUCT_ID,PRODUCT_NAME,CATEGORY,BRAND,ADDRESS_ID,STREET,CITY,STATE,ZIP_CODE,TRANSACTION_ID,QUANTITY_OF_ITEMS_SOLD,UNIT_PRICE,DISCOUNT
195,49,Client_49,Lastname_49,client_49.lastname_49@example.com,105,Store_105,Location_105,205,Product_205,Electronics,Brand_205,310,Street_310,City_310,State_310,77551,1181,4,28.059134,0.07
196,49,Client_49,Lastname_49,client_49.lastname_49@example.com,106,Store_106,Location_106,208,Product_208,Home,Brand_208,302,Street_302,City_302,State_302,78810,1016,5,23.045865,0.17
197,49,Client_49,Lastname_49,client_49.lastname_49@example.com,108,Store_108,Location_108,201,Product_201,Home,Brand_201,305,Street_305,City_305,State_305,57673,1054,6,39.085894,0.26
198,49,Client_49,Lastname_49,client_49.lastname_49@example.com,110,Store_110,Location_110,205,Product_205,Fashion,Brand_205,301,Street_301,City_301,State_301,96208,1021,7,83.685019,0.01
199,50,Client_50,Lastname_50,client_50.lastname_50@example.com,107,Store_107,Location_107,204,Product_204,Electronics,Brand_204,309,Street_309,City_309,State_309,41942,1005,3,13.961897,0.24


In [14]:
transactions3.to_csv("transactions_cleaned.csv")


## Part II: Building a star schema and extracting metrics with SQL

### Creating the tables

First of all, let's take a look at the imported cleaned table, and correct the identifier so it starts with 1

```sql
SELECT *
FROM TRANSACTIONS_CLEANED;

UPDATE transactions_cleaned
SET identifier = identifier + 1;

Now, we will make the dimension and fact tables taking the columns from the whole dataset

```sql
CREATE OR REPLACE TABLE Client AS 
SELECT identifier, client_id, client_name, client_lastname, email
FROM transactions_cleaned;

CREATE OR REPLACE TABLE Store AS 
SELECT identifier, store_id, store_name, location
FROM transactions_cleaned;

CREATE OR REPLACE TABLE Product AS 
SELECT IDENTIFIER, product_id, product_name, category, brand
FROM transactions_cleaned;

CREATE OR REPLACE TABLE Address AS 
SELECT IDENTIFIER, address_id, street, city, state, zip_code
FROM transactions_cleaned;

CREATE OR REPLACE TABLE Sales AS 
SELECT IDENTIFIER, client_id, store_id, product_id, address_id, transaction_id, quantity_of_items_sold, unit_price, discount
FROM transactions_cleaned;

Now that we have all the tables we need, we can query some metrics from them

### Extracting metrics

Having the necessary tables, we can write queries with JOINs and perform operations to extract valuable information from the dataset:

```sql

--Total sales
SELECT SUM(quantity_of_items_sold*unit_price) AS total_revenue, SUM(quantity_of_items_sold*unit_price*(1-discount)) AS total_net_revenue
FROM sales;


-- Number of Clients
SELECT COUNT(DISTINCT client_id) AS num_client
FROM client;

-- Transactions per client
SELECT client_id, COUNT(DISTINCT transaction_id) AS num_transactions
FROM Sales
GROUP BY client_id
ORDER BY num_transactions DESC;

-- Total spend per client
SELECT client_id, SUM(quantity_of_items_sold * unit_price) AS total_spend
FROM Sales
GROUP BY client_id
ORDER BY total_spend DESC;


-- Sales by store
SELECT COUNT(DISTINCT store_id) AS num_stores
FROM Store;

SELECT st.store_id, st.store_name, SUM(quantity_of_items_sold * unit_price) AS total_sales_amount
FROM Sales s
JOIN Store st ON s.identifier = st.identifier
GROUP BY st.store_id, st.store_name
ORDER BY total_sales_amount DESC;


-- Sales by product
SELECT DISTINCT product_id, product_name
FROM product;

SELECT p.product_id, p.product_name, SUM(s.quantity_of_items_sold) AS total_quantity_sold
FROM Sales s
JOIN Product p ON s.identifier = p.identifier
GROUP BY p.product_id, p.product_name
ORDER BY total_quantity_sold DESC
LIMIT 5;


-- Sales by category
SELECT p.category, SUM(s.quantity_of_items_sold * s.unit_price) AS total_sales_amount
FROM Sales s
JOIN Product p ON s.identifier = p.identifier
GROUP BY p.category
ORDER BY total_sales_amount DESC;


-- Sales by cities
SELECT a.city, SUM(s.quantity_of_items_sold * s.unit_price) AS total_sales_amount
FROM Sales s
JOIN Address a ON s.identifier = a.identifier
GROUP BY a.city
ORDER BY total_sales_amount DESC
LIMIT 5;


-- Sales by state
SELECT DISTINCT state
FROM address;

SELECT a.state, SUM(s.quantity_of_items_sold * s.unit_price) AS total_sales_amount
FROM Sales s
JOIN Address a ON s.identifier = a.identifier
GROUP BY a.state
ORDER BY total_sales_amount DESC;

The queries above allow us to extract important information while filtering according to the relevant categories.

## Part III: Presenting the data

Finally, we can take these insights, metrics, and KPIs and present them in an interactive dashboard with a data visualization solution such as Microsoft's Power BI

**Interactive dashboard**

<a href="https://app.powerbi.com/view?r=eyJrIjoiOGE2NjNlOTQtZWU1Zi00YjQ5LTgwYjEtMWQzMzQyMThmYmY5IiwidCI6ImY2ZjdhNzFkLTE1ZDYtNGUwNC04ZjkzLTYwNzgyNmIzYjhmYyIsImMiOjR9&embedImagePlaceholder=true">
    <img src="https://drive.google.com/thumbnail?id=15lQgKjG5XmadP6pr2W-anwcCzO3eqY_K" alt="Dashboard" width="200">
</a>


From the dashboard, we can see important KPIs such as total and net revenue, while also exploring the income by different categories like the store, product, client, and region.

We can immediately see that the highest the highest revenue was generated by Product 205, and that the Home category performance was less impactful than that of the other two categories. We also see the top ten clients by money spent, and that Client 29 spent the most out of all the clients we have, making it a valuable and top-priority client. Finally, we can see that State 308 was the most profitable one, and we can see how each store performed individually by selecting it in the slicer.


This portfolio project showcases the capabilities of Python, SQL, and Power BI for data analysis, providing insight into my fundamental skills as a data analyst. Thank you for taking the time to review it, and I hope it demonstrates my passion and competence in using data to drive informed decision-making.

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
